In [1]:
#settning up the librairies and base configureation to run 

import numpy as np
np.random.seed(42)
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2    
import pandas as pd
    

In [2]:
def process_data1(file):
    
    X_train = pd.read_csv(file)
    X_train = X_train.sample(frac=1)
    df = X_train.copy()
    
    return df

In [3]:
def prepare_columns(dataset):
    
    dataset['Month'] = pd.Categorical(dataset['Month'])
    monthDummy = pd.get_dummies(dataset['Month'], prefix = 'Month')
    dataset = pd.concat([dataset, monthDummy], axis=1)
    dataset.drop(['Month'], axis=1, inplace= True)


    dataset['OperatingSystems'] = pd.Categorical(dataset['OperatingSystems'])
    operatingSystemsDummy = pd.get_dummies(dataset['OperatingSystems'], prefix = 'OS')
    dataset = pd.concat([dataset, operatingSystemsDummy], axis=1)
    dataset.drop(['OperatingSystems'], axis=1, inplace= True)


    dataset['Browser'] = pd.Categorical(dataset['Browser'])
    browserDummy = pd.get_dummies(dataset['Browser'], prefix = 'Browser')
    dataset = pd.concat([dataset, browserDummy], axis=1)
    dataset.drop(['Browser'], axis=1, inplace= True)


    dataset['Region'] = pd.Categorical(dataset['Region'])
    regionDummy = pd.get_dummies(dataset['Region'], prefix = 'Region')
    dataset = pd.concat([dataset, regionDummy], axis=1)
    dataset.drop(['Region'], axis=1, inplace= True)


    dataset['TrafficType'] = pd.Categorical(dataset['TrafficType'])
    trafficTypeDummy = pd.get_dummies(dataset['TrafficType'], prefix = 'TrafficType')
    dataset = pd.concat([dataset, trafficTypeDummy], axis=1)
    dataset.drop(['TrafficType'], axis=1, inplace= True)


    dataset['VisitorType'] = pd.Categorical(dataset['VisitorType'])
    visitorTypeDummy = pd.get_dummies(dataset['VisitorType'], prefix = 'visitorType')
    dataset = pd.concat([dataset, operatingSystemsDummy], axis=1)
    dataset.drop(['VisitorType'], axis=1, inplace= True)


    dataset['Weekend'] = pd.Categorical(dataset['Weekend'])
    weekendDummy = pd.get_dummies(dataset['Weekend'], prefix = 'Weekend')
    dataset = pd.concat([dataset, weekendDummy], axis=1)
    dataset.drop(['Weekend'], axis=1, inplace= True)
        
    dataset.to_csv("PreparedPrediction.csv")
    
      
    #change type
    dataset['Revenue'] = dataset['Revenue'].astype(int)
    
    #converting to binary-valued features
    dataset.ProductRelated[dataset.ProductRelated != 0] = 1
    dataset.ProductRelated[dataset.ProductRelated == 0] = 0

    dataset.Informational[dataset.Informational != 0] = 1
    dataset.Informational[dataset.Informational == 0] = 0

    dataset.Administrative[dataset.Administrative != 0] = 1
    dataset.Administrative[dataset.Administrative == 0] = 0  
    
    dataset.Revenue[dataset.Revenue == True] = 1 
    dataset.Revenue[dataset.Revenue == False] = 0
        
    return dataset

In [4]:
def prepare_X_and_Y(df):
    
    X = df.drop('Revenue', axis = 1)
    Y = df['Revenue'].copy()
    
    return X,Y

In [5]:
def feature_selection(X,Y):
    
    #feature ranking and feature selection
    feature_ranking = SelectKBest(chi2, k=5)
    fit = feature_ranking.fit(X, Y)
    fmt = '%-8s%-20s%s'

    list_one =[]    
    for i, (score, feature) in enumerate(zip(feature_ranking.scores_, X.columns)):
        list_one.append((score, feature))
    dfObj = pd.DataFrame(list_one) 
    dfObj.sort_values(by=[0], ascending = False)

    return dfObj


In [6]:
def select_columns(df):
    
    #columns have been selected based on above top features
    df = df.loc[:, ["Administrative", "Administrative_Duration", "Informational", "Informational_Duration", "ProductRelated", "ProductRelated_Duration", "PageValues", "Revenue"]]
    
    return df


In [7]:
def train_test_datasplit(df):
      
    trainData, testData = train_test_split(df, test_size = 0.3)
    #X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

    
    #X_train = np.array(X_train)
    #X_test = np.array(X_test)
    #X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
    #X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],1))

    #Y_train = np.array(Y_train)
    #Y_train = Y_train.reshape((Y_train.shape[0],2))
    #Y_test = np.array(Y_test)
    #Y_test = Y_test.reshape((Y_test.shape[0],2))
    
    #return X_train, X_test, Y_train, Y_test
    
    return trainData, testData


In [8]:
def saving_datasets(trainData, testData):
    
    trainData.to_csv('LSTMTrainData.csv')
    testData.to_csv('LSTMTestData.csv')
    

In [9]:
df = process_data1('predictionDataset')

df = prepare_columns(df)


X,Y = prepare_X_and_Y(df)

dbObj = feature_selection(X,Y)

df = select_columns(df)

trainData, testData = train_test_datasplit(df)

saving_datasets(trainData, testData)



C:\MyProgramFiles\Anaconda3\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\MyProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
